### Importujemy ważne bilbioteki oraz montujemy dysk google drive




In [ ]:
import os
!pip install biosignalsnotebooks

!apt-get install libmagic-dev

!pip install biosppy
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import butter, filtfilt
from numpy import ediff1d

import biosignalsnotebooks as bsnb

import bokeh
bokeh.io.output_notebook()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 31.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of biosignalsnotebooks to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.3 MB/s eta 

### Wczytujemy wszystkie pliki ekg txt do edycji
Opis plików:

* 1a, 1b, 2a, 2b, 3a, 3b -> EDA na nadgarstku
* 4a, 4b -> EDA połączone na opuszkach palca
* 5 ->EDA i EKG w czasie gry, pomiar EDA na nadgarstku
* 6 -> sam  pomiar EDA na nadgarstku
* 7a 7b, 8a, 8b -> Wykonano pomiary na 2 rozgrywkach. Pomiary wykonano na tej samej osobie, pomiar EDA wykonano na nadgarstku. W pierwszej rozgrywce (pliki "7a, 7b") badanemu nie udało się wygrać w wymaganym czasie, natomiast w drugiej rozgrywce (pliki "8a, 8b") badany wygrał.
* 9a 9b, 10a 10b -> Elektrody do pomiaru EDA przyklejone były powyżej nadgarstka od strony powierzchni dłoniowej, jedna obok drugiej.
wyrzucam 1b.txt bo jest dziwny

In [ ]:
folder_path = "/content/drive/MyDrive/psaw/data/"

raw_ecg_files = [f for f in os.listdir(folder_path)
             if f.endswith(".txt") and os.path.isfile(os.path.join(folder_path, f))]
wristFiles = ['1a','1b','2a','2b','3a','3b']
# eda_files = ['3b.txt','6.txt']
exclude_files = ['6.txt', "5.txt", "3b.txt", "1b.txt"]
# raw_ecg_files = [x for x in raw_ecg_files if x not in eda_files]
raw_ecg_files = [x for x in raw_ecg_files if x not in exclude_files]
#3b i 6 to eda
for file in raw_ecg_files:
  print(file)

Wyświetalmy za każdym razem nasze pliki aby zobaczyć co można z nich wyciągnąć. Poniżej znajdują się najpotrzebniejsze funkcje

In [1]:
def plotEcg(raw_ecg, time, fileName):
    bf = bokeh.plotting.figure(
      x_axis_label= "time[s]",
      y_axis_label= "raw_ecg[u]",
      width=        1200,
      height=       300,
      title="Dane pobrane z pliku: " + fileName
    )
    bf.line(time, raw_ecg, line_width=2)
    bokeh.plotting.show(bf)

def processData(filePath, fileName):
  rawData, header = bsnb.load(filePath, get_header=True)
  raw_ecg = rawData["CH1"]
  raw_eda = rawData['CH2']
  sr = header["sampling rate"]
  ecg_len = raw_ecg.shape[0]
  time = np.linspace(start=0, stop=ecg_len/sr, num=ecg_len)
  plotEcg(raw_ecg, time, fileName)
  plotEcg(raw_eda, time, fileName)
  return raw_ecg, raw_eda, sr, time, ecg_len, header

def plot_pantompkins(series1, name1, series2, name2, peaks=False):
  colors = bokeh.palettes.brewer['Paired'][3]
  bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='ECG (mV)', width=800, height=250)
  bf.line(time, series1, alpha=0.8, line_width=2, color=colors[0], legend_label=name1)
  if peaks:
    bf.circle(time, series2, alpha=0.8, size=10, color=colors[1], legend_label=name2)
  else:
    bf.line(time, series2, alpha=0.8, line_width=2, color=colors[1], legend_label=name2)
  bf.legend.click_policy="hide"
  bokeh.plotting.show(bf)

def bandpass_filter(signal, lowcut, highcut, fs, order=2):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

def returnMasked(mask, time, signal):
  signal[mask] = 0
  return signal

def convert_to_ecg(raw_signal, header):
  n = header["resolution"][0]
  vcc = 3
  g_ecg = 1000

  ecg = ((((raw_signal)/(2**n))-0.5)*vcc)/g_ecg
  return ecg * 1000

def convert_to_eda(raw_signal):
  n = header["resolution"][-1]
  vcc = 3.3
  g_eda = 0.132

  return ((raw_signal / 2**n) * vcc) / g_eda

def moving_heart_rate(peaks_vector, sampling_rate=1000, window_sec=10, step_sec=1):

    peak_locs = np.where(~np.isnan(peaks_vector))[0]
    total_len = len(peaks_vector)
    win_size = int(window_sec * sampling_rate)
    step_size = int(step_sec * sampling_rate)

    times = []
    bpm_values = []

    for start in range(0, total_len - win_size, step_size):
        end = start + win_size
        beat_count = np.sum((peak_locs >= start) & (peak_locs < end))
        bpm = (beat_count / window_sec) * 60
        times.append((start + end) / 2 / sampling_rate)
        bpm_values.append(bpm)

    return np.array(times), np.array(bpm_values)

#maski
#2b.txt
#9b.txt
#10b.txt
#7b.txt
#8b.txt
#5.txt
#4b.txt
masks = [
    lambda time: time <= 150,
    lambda time: False,
    lambda time: not ((time >= 10 and time <= 215) or (time >= 225 and time <= 245)),
    lambda time: False,
    lambda time: False,
    lambda time: False,
    lambda time: False,
]
mask = masks[1]
# for file, mask in raw_ecg_files, masks:
file = raw_ecg_files[1]
raw_ecg, raw_eda, sr, time, ecg_len, header = processData(folder_path + file, file)

NameError: name 'raw_ecg_files' is not defined

## Jak widać musimy zamienić nasze dane z jakichś jednostek z biosignalPlus na miliVolty, żeby jakkolwiek zobaczyć co się tam dzieje, do tego użyjemu naszej funkcji `convert_to_ecg` oraz używamy `mask` aby wyrzucić potencjalny szum i dane do wyrzucenia

In [ ]:
converted_ecg = convert_to_ecg(raw_ecg, header)

combinedMask = np.array([mask(t) for t in time])
converted_ecg = returnMasked(combinedMask, time, converted_ecg)

bf = bokeh.plotting.figure(
    x_axis_label= "time [s]",
    y_axis_label= "converted_ecg [mV]",
    width=1200,
    height=300
)
bf.line(time, converted_ecg, line_width=2)
bokeh.plotting.show(bf)


## Po utworzeniu wykresów i wyświetleniu naszych sygnałów EKG musimy wykorzystać algorytm Pan-Tompkinsa, aby znaleźć nasze załamki, czyli potrzebne informacje -> po kolei wykonujemy, filtr częstotliwości, wykonujemy pochodną sygnału, podnosimy nasz sygnał do drugiej potęgi, całkujemy sygnał i pobieramy nasze załamki

In [ ]:
filtered_ecg = bandpass_filter(converted_ecg, lowcut=5, highcut=15, fs=sr)

plot_pantompkins(converted_ecg, "ECG signal", filtered_ecg, "Filtered ECG")

differentiated_ecg = np.ediff1d(filtered_ecg)

plot_pantompkins(filtered_ecg, "Filtered ECG", differentiated_ecg, "Differentiated ECG")

squared_ecg = differentiated_ecg ** 2

plot_pantompkins(differentiated_ecg, "Differentiated ECG", squared_ecg, "Squared ECG")

window_size = int(0.15 * sr)
window = np.ones(window_size)
integrated_ecg = np.convolve(squared_ecg, window, mode='same')
integrated_ecg /= window_size # Normalization

plot_pantompkins(squared_ecg, "Squared ECG", integrated_ecg, "Integrated ECG")

peaks_found = np.empty(len(converted_ecg))
peaks_found[:] = np.nan

threshold = 1

for x in range(1, len(integrated_ecg)-1):
    if integrated_ecg[x-1] < integrated_ecg[x] > integrated_ecg[x+1]:
        if converted_ecg[x] > threshold:
            peaks_found[x] = converted_ecg[x]

plot_pantompkins(converted_ecg, "ECG signal", peaks_found, "Peaks found", peaks=True)


# Obróbka EDA

In [ ]:
eda_len = raw_eda.shape[0]

In [ ]:
p = bokeh.plotting.figure(title="EDA Signal", x_axis_label='Time (s)', y_axis_label='EDA', width=800, height=400)

p.line(time, raw_eda, line_width=2, color="navy", legend_label="EDA")

bokeh.plotting.show(p)

In [ ]:
converted_eda = convert_to_eda(raw_eda)

# 1μS = 1 / 1MR

bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='EDA (μS)', width=800, height=250)
bf.line(time, converted_eda)
bokeh.plotting.show(bf)

In [ ]:
cutoff = 30 # Hz
b, a = signal.butter(2, cutoff, fs=sr, btype='lowpass')
wide_filtered_eda = signal.filtfilt(b, a, converted_eda)

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(converted_eda, label="Original EDA")
plt.plot(wide_filtered_eda, label="Wide Filtered EDA (0–30 Hz)")
plt.xlabel("Time (samples)")
plt.ylabel("EDA (µS)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Z uwagi na ograniczenia algorytmu usuwającego artefakty, oraz (wynikające z porównania) znikome różnice w wyniku algorytmu w zestawieniu z oryginalnym EDA, postanowiliśmy przejść odrazu do wygładzenia sygnału.

In [ ]:
window_size = 0.3 * sr
window = np.ones(int(window_size)) / window_size

smoothed_eda = np.convolve(wide_filtered_eda, window, mode='valid')

smoothed_eda = smoothed_eda * (np.max(wide_filtered_eda) / np.max(smoothed_eda))

bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Normalized EDA', width=1600, height=700)
bf.line(time, wide_filtered_eda/np.max(wide_filtered_eda), alpha=0.4, line_width=2, color='red', legend_label="Lowpass (30 Hz)")
bf.line(time, smoothed_eda/np.max(smoothed_eda), alpha=0.4, line_width=2, color='blue', legend_label="Smoothed EDA")
bf.legend.click_policy="hide"
bokeh.plotting.show(bf)

In [ ]:
!pip install biosppy
!pip install peakutils

import biosppy

In [ ]:
results_eda = biosppy.signals.eda.eda(signal=smoothed_eda, sampling_rate=sr, show=True, min_amplitude=0.01)

In [ ]:
phasic_eda = results_eda[2]
tonic_eda = results_eda[3]

bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Signal Amplitude', width=800, height=300)
bf.line(time, tonic_eda, alpha=0.7, line_width=2, color='blue', legend_label="Tonic EDA")
bf.legend.click_policy="hide"
bokeh.plotting.show(bf)

# Nakładamy nasze dane z EDA oraz EKG

In [ ]:
tonic_eda = tonic_eda[:200000]
eda = smoothed_eda[:200000]
ecg = converted_ecg[:200000]
peaks = peaks_found[:200000]
t = time[:200000]

# Z uwagi na szumy w ECG, końcowe kilkanaście sekund sygnału zostało obcięte

bf = bokeh.plotting.figure(x_axis_label='Time (s)', y_axis_label='Signal Amplitude', width=1200, height=500)
bf.line(time[7000:27000], ecg[7000:27000], alpha=0.7, line_width=2, color='red', legend_label="Converted ECG")
bf.line(time[7000:27000], eda[7000:27000], alpha=0.7, line_width=2, color='navy', legend_label="Smoothed EDA")
bf.scatter(time[7000:27000], peaks[7000:27000], alpha=0.7, line_width=2, color='maroon', legend_label="Peaks")
bf.legend.click_policy="hide"
bokeh.plotting.show(bf)

In [ ]:
times, hr_values = moving_heart_rate(peaks, sampling_rate=1000, window_sec = 15, step_sec=0.7)

from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d

smoothed_bpm = gaussian_filter1d(hr_values, sigma=2)

interp_func = interp1d(times, smoothed_bpm, kind='linear', bounds_error=False, fill_value="extrapolate")
smoothed_bpm_full = interp_func(t)

smoothed_bpm_full[t < times[0]] = smoothed_bpm[0]
smoothed_bpm_full[t > times[-1]] = smoothed_bpm[-1]

plt.figure(figsize=(12, 4))
plt.plot(times, hr_values, c='pink')
plt.plot(t, smoothed_bpm_full, c='magenta')
plt.plot(t, tonic_eda * 13, c='navy')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Moving Heart Rate Estimate')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import pearsonr

mask = ~np.isnan(tonic_eda) & ~np.isnan(smoothed_bpm_full)
x = tonic_eda[mask]
y = smoothed_bpm_full[mask]

# Optionally normalize
# x = (x - np.mean(x)) / np.std(x)
# y = (y - np.mean(y)) / np.std(y)

r, p = pearsonr(x[:50000], y[:50000])

print(f"Pearson correlation coefficient (r): {r:.4f}")
print(f"P-value: {p:.4g}")

# Wnioski

Na początkowym etapie badania dane tętna oraz EDA mocno ze sobą korelowały, co wskazuje na istnienie powiązania między emocjami o podwyższeniem akcji serca, oraz zwiększeniem przewodniości skóry.